In [2]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect

# Configurações de conexão
connection_string = 'postgresql://postgres:a@localhost:5432/postgres'
engine = create_engine(connection_string)

def copiar_schema_automaticamente():
    # 1. Criar o inspetor para ler metadados do banco
    inspector = inspect(engine)
    
    # 2. Obter automaticamente todos os nomes de tabelas do schema 'cvm'
    tabelas = inspector.get_table_names(schema='cvm')
    
    if not tabelas:
        print("Nenhuma tabela encontrada no schema 'cvm'.")
        return

    print(f"Encontradas {len(tabelas)} tabelas. Iniciando cópia...")

    with engine.connect() as conn:
        # Garantir que o schema de destino exista
        conn.execute(text("CREATE SCHEMA IF NOT EXISTS cvm_copia;"))
        conn.commit()
        
        for tabela in tabelas:
            print(f"Processando: {tabela}...")
            
            try:
                # Ler a tabela original
                df = pd.read_sql_table(tabela, conn, schema='cvm')
                
                # Copiar para o novo schema
                df.to_sql(
                    tabela, 
                    conn, 
                    schema='cvm_copia', 
                    if_exists='replace', 
                    index=False,
                    method='multi', # Acelera a inserção
                    chunksize=10000  # Evita estouro de memória em tabelas grandes
                )
                print(f"   -> OK!")
            
            except Exception as e:
                print(f"   -> Erro ao copiar {tabela}: {e}")
copiar_schema_automaticamente()

Encontradas 165 tabelas. Iniciando cópia...
Processando: agente_auton_cad_agente_auton_pj...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta_bpp_con...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta_dva_con...
   -> OK!
Processando: adm_cart_cad_adm_cart_diretor...
   -> OK!
Processando: adm_cart_cad_adm_cart_pf...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta_bpa_con...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta_dfc_mi_ind...
   -> OK!
Processando: cia_aberta_doc_fca_cia_aberta...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta_dra_con...
   -> OK!
Processando: adm_cart_cad_adm_cart_pj...
   -> OK!
Processando: adm_cart_cad_adm_cart_resp...
   -> OK!
Processando: adm_cart_cad_adm_cart_socios...
   -> OK!
Processando: adm_fii_cad_adm_fii...
   -> OK!
Processando: agente_auton_cad_agente_auton_pf...
   -> OK!
Processando: cia_aberta_doc_dfp_cia_aberta_bpp_ind...
   -> OK!
Processando: cia_aberta_doc